# LLMs and Chat Models

In [ ]:
# text-davinci-003
from langchain.llms.openai import OpenAI
# gpt-r.5-turbo
from langchain.chat_models import ChatOpenAI
llm = OpenAI()
chat = ChatOpenAI()

# predict just one string, message
a = llm.predict("How many planets are there")
b = chat.predict("How many planets are there")
a,b

# Predict Messages

In [ ]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.1)

In [ ]:
# predict list of messages
# message constructors
# 맥락 및 조건 설정 가능, 작성한 것들 memory에 추가, 
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content = "You are a georgraphy expoert. And you only reply in Inalian.",
    ),
    AIMessage(content= "Ciao, mi chiamo Paolo"),
    HumanMessage(
        content = "What is the distance between Mexico and Thailand. Also, what isyour name?",
    ),
]
chat.predict_messages(messages)

# Prompt Template

In [ ]:
# 템플릿은 일종의 validation 기능을 해서 쓸대없는 문장을 소비하지 않게 만들어줌
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}."
)
prompt = template.format(country_a = "Mexico",country_b = "Thailand")
chat.predict(prompt)


In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        ('system',"You are a georgraphy expoert. And you only reply in {language}."),
        ("ai","Ciao, mi chiamo {name}"),
        ("human","What is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)
prompt = template.format_messages(
    language = 'Greek',
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand"

)
chat.predict_messages(prompt)

# Output Parser and LangChainExpressionLanguage(LCEL)

In [ ]:
# LangChain Expression Language
from langchain.schema import BaseOutputParser

# LLM 응답을 변형해야 할 때 사용하기 위해서
class CommaOutputParser(BaseOutputParser):
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
p = CommaOutputParser()
p.parse("Hello,how, are, you")
    

In [ ]:
from langchain.prompts import ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ('system', "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else"),
    ("human","{question}")
]
)
prompt = template.format_messages(max_items = 10,question = "What are the planets?")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

In [ ]:
prompt = template.format_messages(max_items = 10, question = "What are the colors")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

- 체인을 이용하면 각각의 predict 결과를 연결해서 사용할 수 있음
- R의 chain 같은 느낌
```
chain1 = template1 | chat | outputparser1
chain2 = template2 | chat | outputparser2
chain_sum = chain1 | chain2 | outputparser_sum
```

In [ ]:
# 코드가 너무 김
# 기존 작업을 위해 작성한 코드: Chat model, 파서 생성, 템플릿 생성, message format, predict, 파서 호출, 파싱 
template = ChatPromptTemplate.from_messages([
    ('system', "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else"),
    ("human","{question}")
]
)
# chain 사용
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the poketmons?"
})

# Chaining Chains
- [invoke 설명](https://python.langchain.com/docs/expression_language/interface)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
# Streaming은 모델의 응답이 생성되는걸 보게 해줌, 
chat = ChatOpenAI(
    temperature=0.1,streaming=True,callbacks=[StreamingStdOutCallbackHandler()]
    )


chef_template = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food")
])

chef_chain = chef_template | chat 

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetereian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    "human","{recipe}"
])
veg_chain = veg_chef_prompt | chat


final_chain = {"recipe": chef_chain} | veg_chain


final_chain.invoke({
    "cuisine":"indian"
})


# [Module format](https://python.langchain.com/docs/modules/)
- Model I/O
    - Model Input & Output
    - prompts, language model, output parser


- Retrieval
    - 외부데이터를 모델에 어떻게 적용하느냐

- Chains

- Memory

- Agents


# Fewshot Prompt Template
- prompt template을 디스크에 저장하고 로드할 수 있기 때문에 필요함

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

t = PromptTemplate(
    template = "What is the capital of {country}",
    input_variables=["country"],
)
t.format(country = "France")

t= PromptTemplate.from_template("What is the capital of {country}")
t.format(country = "France")


In [ ]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

example_prompt =  PromptTemplate.from_template("Human:{question}\nAI:{answer}")

# suffix 는 사용자의 질문
prompt = FewShotPromptTemplate(
    example_prompt= example_prompt,
    examples = examples,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
    )


# chat.predict(prompt.format(country = 'Germany'))
chain = prompt | chat
chain.invoke({
    "country": "Turkey"
})

# Few Shot Chat Message Prompt Template

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example prompt는 example과 key값이 일치해야함
example_prompt =  ChatPromptTemplate.from_messages(
    [
    ("human", "What do you know about {country}?"),
    ("ai","{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt= example_prompt,
    examples = examples,
    )

final_prompt = ChatPromptTemplate.from_messages(
    [
    ("system"," You are a geography expert, you gives short answers"),
    example_prompt,
    ("human","What do you know about {country}?"),
    ],
)

chain = final_prompt | chat
chain.invoke({
    "country": "THailand"
})

In [ ]:
print(final_prompt.format(country = "Germany"))

# Length Based Example Selector


In [88]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


        


chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self,examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

    
example_prompt =  PromptTemplate.from_template("Human:{question}\nAI:{answer}")

# example_selector = LengthBasedExampleSelector(
#     examples = examples,
#     example_prompt = example_prompt,
#     max_length = 80)

example_selector = RandomExampleSelector(
    examples = examples,
)

prompt = FewShotPromptTemplate(
    example_prompt= example_prompt,
    example_selector= example_selector,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
    )

prompt.format(country='Brazil')

'Human:What do you know about Greece?\nAI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

# Serialize
- 디스크에서 프롬프트를 가져오는 법
- 이를 통해 prompt를 별도로 저장 가능함


In [91]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate

prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

prompt.format(country = "Germany")


'What is the capital of Germany'

chat = ChatOpenAI(
    temperature = 0.1,
    streaming= True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)


prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]


full_prompt = PipelinePromptTemplate(
    final_prompt = final,
    pipeline_prompts= )